# Install .NET and Tools

1. [.NET Framework](https://dot.net/learntocode)
2. [.NET Interactive Notebooks Extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode)
3. [Data Table Extension](https://marketplace.visualstudio.com/items?itemName=RandomFractalsInc.vscode-data-table)

# Useful videos to watch regarding .NET Interactive Notebooks

1. [Learn C# with Interactive Notebooks](https://www.youtube.com/watch?v=xdmdR2JfKfM)
2. [NET Interactive Notebooks with C#/F# in VS Code](https://www.youtube.com/watch?v=DMYtIJT1OeU)
3. [.NET Everywhere - Windows, Linux, and Beyond](https://www.youtube.com/watch?v=ZM6OO2lkxA4)

# Install Power Fx kernel

In [ ]:
#r "nuget: PowerFx.NET.Interactive,2021.12.3.1"

# Set Variable

In [ ]:
#!powerfx
Set(environment, ".NET Interactive")

# Set Table

In [ ]:
#!powerfx
ClearCollect( Products,
    Table(
        { Product: "Widget",    'Quantity Requested': 6,  'Quantity Available': 3 },
        { Product: "Gadget",    'Quantity Requested': 10, 'Quantity Available': 20 },
        { Product: "Gizmo",     'Quantity Requested': 4,  'Quantity Available': 11 },
        { Product: "Apparatus", 'Quantity Requested': 7,  'Quantity Available': 6 }
    )
);

# Run Multiple Formulas

In [ ]:
#!powerfx
Set(varInput, ForAll( ["4"], Sqrt( Value ) ));
Sequence(4);
First(varInput);

# Filtering

In [ ]:
#!powerfx
ClearCollect( Products,
    Table(
        { Product: "Widget",    'Quantity Requested': 6,  'Quantity Available': 3 },
        { Product: "Gadget",    'Quantity Requested': 10, 'Quantity Available': 20 },
        { Product: "Gizmo",     'Quantity Requested': 4,  'Quantity Available': 11 },
        { Product: "Apparatus", 'Quantity Requested': 7,  'Quantity Available': 6 }
    )
);
First(Products);
AddColumns( Filter( Products, 'Quantity Requested' > 'Quantity Available' ), "Quantity To Order", 'Quantity Requested' - 'Quantity Available');

# Query current state

In [ ]:
#!powerfx
?

# Create variables in C#

In [ ]:
using System.Collections.Generic;

var x = new List<string>(new[]{"a"});
var today = DateTime.Now;

# Use variables from C# in Power Fx

In [ ]:
#!powerfx
#!share x --from csharp
#!share today --from csharp
Set(varAlphabets, x);
Set(varToday, today);

# Set variables in Power Fx

In [ ]:
#!powerfx
Set(varText, "Hello");

# Use variable from Power Fx in C#

In [ ]:
#!share varText --from powerfx
var output = varText;
Console.WriteLine(output);

# Creating new Functions in Power Fx

You can create new Functions by extending a new class from `ReflectionFunction`. The classname should end with `Function`. Let us implement three Functions:

1. Slice - This is similar to JavaScript `Array.slice` function. It accepts 3 parameters: The Table to slice records from, starting index of the slice, and number of items to slice. You can pass `Blank`, if you want all the items from the starting index of slice.
2. Keys - This is similar to `Object.keys` function. It returns all the Object (`RecordValue`) values as a Table.
3. Zip - This Function combines two Tables and returns a new Table with all properties in both the Tables.

In [ ]:
using Microsoft.PowerFx;
using Microsoft.PowerFx.Core.Functions;
using Microsoft.PowerFx.Core.Localization;
using Microsoft.PowerFx.Core.Public.Types;
using Microsoft.PowerFx.Core.Public.Values;
using Microsoft.PowerFx.Core.Types;
using Microsoft.PowerFx.Core.Utils;
using PowerFxDotnetInteractive;

private class SliceFunction : ReflectionFunction
{
    //2nd argument to base constructor is return type of Execute method. The subsequent ones are the arguments to the Execute method.
    public SliceFunction() : base("Slice", new TableType(), new TableType(), FormulaType.Number, FormulaType.Number)
    {
    }

    public TableValue Execute(TableValue source, NumberValue skipCount, FormulaValue takeCount)
    {
        if(takeCount is NumberValue)
            return FormulaValue.TableFromRecords(source.Rows.Skip((int)skipCount.Value).Select(x=>x.Value).Take((int)(takeCount as NumberValue).Value).ToArray());
        return FormulaValue.TableFromRecords(source.Rows.Skip((int)skipCount.Value).Select(x=>x.Value).ToArray());
    }    
}

private class KeysFunction : ReflectionFunction
{
    //2nd argument to base constructor is return type of Execute method. The subsequent ones are the arguments to the Execute method.
    public KeysFunction() : base("Keys", new TableType(), new RecordType())
    {
    }

    public TableValue Execute(RecordValue source) => FormulaValue.NewTable(source.Fields.Select(x => x.Name));
}

private class ZipFunction : ReflectionFunction
{
    //2nd argument to base constructor is return type of Execute method. The subsequent ones are the arguments to the Execute method.
    public ZipFunction() : base("Zip", new TableType(), new TableType(), new TableType())
    {
    }

    public TableValue Execute(TableValue source1, TableValue source2)
    {
        //zip both the arrays
        var zippedArray = source1.Rows.Zip(source2.Rows, 
            (x, y) => {
                return TableValue.RecordFromFields(Enumerable.Concat(x.Value.Fields, y.Value.Fields));
            });
        return FormulaValue.NewTable(zippedArray);        
    }
}

Now let us add the function to the Power Fx Engine. We obtain the instance of `RecalcEngine` from the Power Fx kernel and add our custom Function.

In [ ]:
PowerFxKernel.GetRecalcEngine().AddFunction(new SliceFunction());
PowerFxKernel.GetRecalcEngine().AddFunction(new KeysFunction());
PowerFxKernel.GetRecalcEngine().AddFunction(new ZipFunction());

We can verify that our new Functions have been added to the list of available Functions.

In [ ]:
PowerFxKernel.GetRecalcEngine().GetAllFunctionNames().Where(x=>x == "Keys" || x == "Slice" || x == "Zip")

Now that our new Functions have been added to the `RecalcEngine`, let us use them.

## Slice Function

In [ ]:
#!powerfx
Set(MovieTable, Table({ Name: "Iron Man", Rating: 10 }, { Name: "Captain America", Rating: 10 }, { Name: "Dark Knight", Rating: 10 }));
Slice(MovieTable, 1, 1);
Slice(MovieTable, 1, Blank());

## Keys Function

In [ ]:
#!powerfx
Set(varPerson, { FirstName: "Max", LastName: "Power" });
Keys(varPerson);

## Zip Function

In [ ]:
#!powerfx
Set(CategoryTable, Table({ Category: "Fruit", Color: "Red" }, { Category: "Vegetable", Color: "Green" }));
Set(FruitAndVegTable, Table({ Name: "Apple" },{ Name: "Cucumber" }));
Zip(CategoryTable, FruitAndVegTable);